### This notebook creates interferorgams within shared footprints of two bursts within the San Joaquin Valley, CA and saves to GeoTIFFS.

Load notebook dependencies.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os, glob
import h5py
import fsspec
import pandas as pd
from pyproj import Proj, CRS
import matplotlib.pyplot as plt
import shapely.wkt as wkt
from shapely import geometry, ops
import xarray as xr
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin
import rioxarray
import sys
sys.path.append('../../')
from src.cslc_utils import get_s3path, read_cslc, cslc_info, rasterWrite, custom_merge, colorize, getbasemaps, moving_window_mean

import warnings
warnings.filterwarnings('ignore')

We will be streaming CSLC-S1 products from S3, which requires the paths in `T144_308014_IW.txt` and `T144_308015_IW.txt`.

In [3]:
data_dir = 's3://opera-pst-rs-pop1/products/CSLC_S1'
burst_id = ['T144_308014_IW2', 
            'T144_308015_IW2']

# get list of dates
b1paths = (open(f"{burst_id[0]}.txt","r").read().split("\n"))
b2paths = (open(f"{burst_id[1]}.txt","r").read().split("\n"))
dates = [path[33:41] for path in b1paths]
nd = len(dates)

# send to dataframe
file_df = pd.DataFrame({'Date':dates,f'{burst_id[0]}':b1paths,f'{burst_id[1]}':b2paths})

# Check if dir to store tiffs exists and make if not
if os.path.exists("../TIFFs/ifgs") == False:
    os.mkdir('../TIFFs/ifgs')

Loop through all available dates, create sequential inteferograms, and save to GeoTIFFs.

In [5]:
# get paths of CSLCs and open files
b1path1_h5 = file_df[f'{burst_id[0]}'][0][:-1]
b2path1_h5 = file_df[f'{burst_id[1]}'][0][:-1]

b1cslc1 = read_cslc(f'{data_dir}/{b1path1_h5}/{b1path1_h5}.h5')
b2cslc1 = read_cslc(f'{data_dir}/{b2path1_h5}/{b2path1_h5}.h5')

for i in np.arange(nd-1):
    b1path2_h5 = file_df[f'{burst_id[0]}'][i+1][:-1]
    b1cslc2 = read_cslc(f'{data_dir}/{b1path2_h5}/{b1path2_h5}.h5')
    b1ifg = b1cslc1*np.conj(b1cslc2)

    b2path2_h5 = file_df[f'{burst_id[1]}'][i+1][:-1]
    b2cslc2 = read_cslc(f'{data_dir}/{b2path2_h5}/{b2path2_h5}.h5')
    b2ifg = b2cslc1*np.conj(b2cslc2)

    # coordinates of each burst
    xcoor1, ycoor1, dx, dy, epsg, bounding_polygon, orbit_direction = cslc_info(f'{data_dir}/{b1path2_h5}/{b1path2_h5}.h5')
    xcoor2, ycoor2, dx, dy, epsg, bounding_polygon, orbit_direction = cslc_info(f'{data_dir}/{b2path2_h5}/{b2path2_h5}.h5')

    # shared coordinates between burtsts
    xvalues, xind1, xind2 = np.intersect1d(xcoor1,xcoor2,return_indices=True)
    xind1 = sorted(xind1)
    xind2 = sorted(xind2)
    yvalues, yind1, yind2 = np.intersect1d(ycoor1,ycoor2,return_indices=True)
    yind1 = sorted(yind1)
    yind2 = sorted(yind2)
    
    # crop to burst overlap
    crop_b1 = b1ifg[yind1[0]:yind1[-1]+1,xind1[0]:xind1[-1]+1]
    crop_b2 = b2ifg[yind2[0]:yind2[-1]+1,xind2[0]:xind2[-1]+1]
    mask1 = np.isnan(crop_b1)
    mask2 = np.isnan(crop_b2)
    crop_b1[mask2] = np.nan
    crop_b2[mask1] = np.nan
    
    # get phase of each interferogram
    phase1 = np.angle(crop_b1)
    phase2 = np.angle(crop_b2)

    # save as GeoTIFFs
    filename1 = f'IFG_{burst_id[0]}_{dates[i]}_{dates[i+1]}.tif'
    filename2 = f'IFG_{burst_id[1]}_{dates[i]}_{dates[i+1]}.tif'
    # Save each interferogram's phase as GeoTiff
    transform = from_origin(xvalues[0],yvalues[-1],dx,-dy)
    rasterWrite('../TIFFs/ifgs/'+filename1,phase1,transform,epsg,dtype='float32')
    rasterWrite('../TIFFs/ifgs/'+filename2,phase2,transform,epsg,dtype='float32')

    b1cslc1 = b1cslc2
    b2cslc1 = b2cslc2  

Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1_T144-308014-IW2_20150212T135942Z_20230929T093447Z_S1A_VV_v1.0/OPERA_L2_CSLC-S1_T144-308014-IW2_20150212T135942Z_20230929T093447Z_S1A_VV_v1.0.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1_T144-308015-IW2_20150212T135945Z_20230929T093419Z_S1A_VV_v1.0/OPERA_L2_CSLC-S1_T144-308015-IW2_20150212T135945Z_20230929T093419Z_S1A_VV_v1.0.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1_T144-308014-IW2_20150401T135943Z_20230929T093321Z_S1A_VV_v1.0/OPERA_L2_CSLC-S1_T144-308014-IW2_20150401T135943Z_20230929T093321Z_S1A_VV_v1.0.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1_T144-308015-IW2_20150401T135946Z_20230929T093342Z_S1A_VV_v1.0/OPERA_L2_CSLC-S1_T144-308015-IW2_20150401T135946Z_20230929T093342Z_S1A_VV_v1.0.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1_T144-308014-IW2_20150425T135944Z_20230929T093248Z_S1A_VV_v1.0/OPERA_L2_CSLC-S1_T144-